In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [4]:
# Load the data
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
# Preprocess the data
# Dropping the irrelevant columns for prediction [RowNumber, CustomerId, Surname]
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [ ]:
# Encoding the categorical variable[Gender]
label_encoder = LabelEncoder()
data['Gender'] = label_encoder.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [9]:
data['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [55]:
# One hot encoding the categorical variable[Geography]
from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder()
onehot_encoder_geo = onehot_encoder.fit_transform(data[['Geography']])
onehot_encoder_geo



<10000x3 sparse matrix of type '<class 'numpy.float64'>'
	with 10000 stored elements in Compressed Sparse Row format>

In [58]:
onehot_encoder_geo.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [59]:
feature_names = onehot_encoder.get_feature_names_out(['Geography'])
feature_names  # ['Geography_France', 'Geography_Spain', 'Geography_Germany']

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [61]:
encoded_df_geo = pd.DataFrame(onehot_encoder_geo.toarray(),columns=feature_names)
encoded_df_geo

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [62]:
# combine the original data with the encoded data
data = pd.concat([data.drop('Geography',axis=1), encoded_df_geo], axis=1)
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [64]:
# Save the encoder and scaler
with open('label_encoder.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)

with open('onehot_encoder.pkl', 'wb') as file:
    pickle.dump(onehot_encoder, file)

In [67]:
# Divide the data into features and target
X = data.drop('Exited', axis = 1)
y = data['Exited']

# Split the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)
          

## ANN Implementation

In [69]:
!pip install tensorflow

  Using cached Markdown-3.7-py3-none-any.whl.metadata (7.0 kB)
   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   ---------------------------------------- 3.4/390.3 MB 20.0 MB/s eta 0:00:20
   - -------------------------------------- 10.2/390.3 MB 26.6 MB/s eta 0:00:15
   - -------------------------------------- 13.9/390.3 MB 24.2 MB/s eta 0:00:16
   - -------------------------------------- 16.5/390.3 MB 21.2 MB/s eta 0:00:18
   - -------------------------------------- 17.0/390.3 MB 17.6 MB/s eta 0:00:22
   - -------------------------------------- 17.6/390.3 MB 14.4 MB/s eta 0:00:26
   - -------------------------------------- 18.1/390.3 MB 12.4 MB/s eta 0:00:31
   - -------------------------------------- 18.6/390.3 MB 11.3 MB/s eta 0:00:33
   - -------------------------------------- 19.1/390.3 MB 10.4 MB/s eta 0:00:36
   -- ------------------------------------- 19.9/390.3 MB 9.6 MB/s eta 0:00:39
   -- ------------------------------------- 20.7/390.3 MB 9.0 MB/s e

In [74]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [75]:
(X_train.shape[1],)

(12,)

In [ ]:
# Build ANN Model
model = Sequential(
    [
        Dense(64, activation='relu',input_shape=(X_train.shape[1],)), #HL1
        Dense(32, activation='relu'),                                 #HL2
        Dense(1, activation='sigmoid')                                #Output Layer
    ]
)

c:\Users\Pooja\miniconda3\envs\udemy\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [79]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# The adam optiimzer comes with a default learning rate and inorder to updatea the learning rate with other values,we can use the following
optimizer1 = tf.keras.optimizers.Adam(learning_rate=0.01)
binaryloss1=tf.keras.losses.BinaryCrossentropy

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# model.complie(optimizer=optimizer1, loss=binary1, metrics=['accuracy'])

In [82]:
# Setup the tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [83]:
# Setting Earlystopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True) # vall_loss is the validation loss

In [84]:
# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test,y_test),
                    validation_split=0.2, 
                    epochs=100, 
                    batch_size=32, 
                    callbacks=[early_stopping, tensorboard_callback])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7464 - loss: 0.5188 - val_accuracy: 0.8330 - val_loss: 0.3844
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8420 - loss: 0.3917 - val_accuracy: 0.8530 - val_loss: 0.3516
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8619 - loss: 0.3432 - val_accuracy: 0.8530 - val_loss: 0.3515
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8686 - loss: 0.3339 - val_accuracy: 0.8590 - val_loss: 0.3413
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8542 - loss: 0.3477 - val_accuracy: 0.8630 - val_loss: 0.3392
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8642 - loss: 0.3331 - val_accuracy: 0.8610 - val_loss: 0.3395
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8688 - loss: 0.3234 - val_accuracy: 0.8590 - val_loss: 0.3395
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8659 - loss: 0.3308 - val_accu

In [85]:
model.save('model.h5')

In [86]:
# Load the TensorBoard Extension
%load_ext tensorboard


In [92]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 5844), started 0:10:59 ago. (Use '!kill 5844' to kill it.)